In [ ]:
# add the grape directory to the path
import sys
sys.path.append('../../qoc')
import grape

In [ ]:
%matplotlib inline
# from matplotlib import interactive
import os
import sys
import inspect
import numpy as np
from scipy.special import factorial
import h5py, scipy
from h5py import File
import matplotlib.pyplot as plt
import matplotlib. cm as cm

from qoc.helper_functions.grape_functions import *
from qoc.main_grape.grape import Grape
from qutip import *  

# Defining the MM Hamiltonian

In the RWA and strong dispersive regime, 
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\tilde{\hat{H}} = \frac{\kappa}{2}\hat{a}^{\dagger}\hat{a}\left(\hat{a}^{\dagger}\hat{a}-1\right) +  2\ket{e}\bra{e}(\chi\hat{a}^{\dagger}\hat{a}+ \frac{\chi_{2}}{2} \hat{a}^{\dagger}\hat{a} \left(\hat{a}^{\dagger}\hat{a}-1\right)) +  \left(\epsilon_{c}(t)+ \epsilon_{q}(t)+\mathrm{c.c.}\right)$$

In [ ]:
class qoc_test_mm:

    def __init__(self, qubit_state_num, mode_state_num, hparams, initial_state = [0], final_state = [3],
                 t1params=None, t2params=None, ROTATING=True, SAMPLE_RATE=1, use_full_H=True):
        
        self.qnum = qubit_state_num #number of qubit levels (g, e, f, h etc.)
        self.mnum = mode_state_num #number of cavity levels
        self.ROTATING = ROTATING
        self.hparams = hparams
        self.initial_state = initial_state
        self.final_state = final_state
        self.t1params = t1params
        self.t2params = t2params
        self.SAMPLE_RATE = SAMPLE_RATE #number of points for mesolve per ns
        self.use_full_H = use_full_H
        
        #Qubit rotation matrices
        self.Q_x = np.diag(np.sqrt(np.arange(1, qubit_state_num)), 1)+np.diag(np.sqrt(np.arange(1, qubit_state_num)), -1)
        self.Q_y = (0+1j) * (np.diag(np.sqrt(np.arange(1, qubit_state_num)), 1)-
                             np.diag(np.sqrt(np.arange(1, qubit_state_num)), -1))
        self.Q_z = np.diag(np.arange(0, qubit_state_num))
        self.I_q = np.identity(qubit_state_num)

        #Cavity rotation matrices
        self.M_x = np.diag(np.sqrt(np.arange(1, mode_state_num)), 1)+np.diag(np.sqrt(np.arange(1, mode_state_num)), -1)
        self.M_y = (0+1j) * (np.diag(np.sqrt(np.arange(1, mode_state_num)), 1)-
                             np.diag(np.sqrt(np.arange(1, mode_state_num)), -1))
        self.M_z = np.diag(np.arange(0, mode_state_num))
        self.I_m = np.identity(mode_state_num)
        
        self.am =  Qobj(np.kron(self.I_q, np.diag(np.sqrt(np.arange(1, mode_state_num)), 1))) #tensor product of the qubit
        #identity with anhilation of the cavity state
        self.aq =  Qobj(np.kron(np.diag(np.sqrt(np.arange(1, self.qnum)), 1), self.I_m ))
        self.sigmaz_q = Qobj(np.kron(self.Q_z, self.I_m)) #z operator on the qubit
        
    def openfile(self, filename):
        
        return File(filename,'r')
    
    def H_rot(self):
        chi, kappa, chi_2 = self.hparams["chi"], self.hparams["kappa"], self.hparams["chi_2"]
        freq_ge, mode_ens = 0, 0 # GHz, in lab frame
        chi_mat = np.zeros(self.qnum)
        chi_2_mat = np.zeros(self.qnum)

        if self.qnum <= 2:
            chi_mat[1] = chi[0] # ge
            chi_2_mat[1] = chi_2[0] #ge

        if self.qnum > 2:
            chi_mat[1] = chi[0] # ge
            chi_mat[2] = chi[1] #ef
            chi_2_mat[1] = chi_2[0] #ge
            chi_2_mat[2] = chi_2[1] #ef
    
        #self-kerr of the cavity modes    
        mode_freq = 0
        mode_ens = np.array([2*np.pi*ii*(mode_freq - 0.5*(ii-1)*kappa) for ii in np.arange(self.mnum)])
        H_m = np.diag(mode_ens)
                
        H0_1 = np.kron(self.I_q, H_m) + 2 * 2 * np.pi * (np.kron(np.diag(chi_mat), self.M_z))
        H0_2 =2*np.pi*np.kron(np.diag(chi_2_mat), np.diag(np.array([ii*(ii-1) for ii in np.arange(self.mnum)])))

        if self.use_full_H:
            return (H0_1 + H0_2)
        
####################################################################################################################
    
    def controlHs(self):
        
        controlHs = []   
        #qubit, cavity rotations 
        XI = np.kron(self.Q_x, self.I_m)
        YI = np.kron(self.Q_y, self.I_m)
        IX = np.kron(self.I_q, self.M_x)
        IY = np.kron(self.I_q, self.M_y)
        
        if self.ROTATING:
            controlHs.append(Qobj(XI))
            controlHs.append(Qobj(YI))         
            controlHs.append(Qobj(IX))
            controlHs.append(Qobj(IY))
        
        return controlHs
    
    def return_pulses(self, filename):
        
        fine_pulses = [] #finer resolution pulses for qutip mesolver
        self.num_ops = len(self.controlHs()) #number of operators (pulses)
        self.f = self.openfile(filename)
        self.total_time = self.f['total_time'][()]
        self.steps = self.f['steps'][()]
        self.dt = float(self.total_time) /self.steps
        self.fine_steps = self.total_time * self.SAMPLE_RATE
        self.base_times = np.arange(self.steps + 1) * self.total_time / self.steps
        self.tlist = np.arange(self.fine_steps + 1) * self.total_time / self.fine_steps #this makes tlist
        #as the class variable which can be accessed by other functions 
        for i in range(self.num_ops):
            base_pulse = self.f['uks'][-1][i]  # final control pulse
            base_pulse = np.append(base_pulse, 0.0)  # add extra 0 on end of pulses for interpolation
            interpFun = scipy.interpolate.interp1d(self.base_times, base_pulse)
            pulse_interp = interpFun(self.tlist)
            fine_pulses.append(pulse_interp) #nopsxfine_steps matrix containing all the pulses
        self.f.close()
        return fine_pulses

    def plot_pulses(self, filename, plot_qubit=True):
    
        pulses = self.return_pulses(filename)
        fig, ax = plt.subplots(nrows=1, figsize=(14,4))
        if plot_qubit: 
            start = 0
        else:
            start = int(2)
        labels = ["Qubit_x", "Qubit_y", "Cavity_x", "Cavity_y"]
        for ii, x in enumerate(pulses[start:]):
            ax.plot(self.tlist/1e3, (x/2/np.pi)*1e3, label=labels[ii+start]) # plotting the linear frequency in MHz
        ax.set_xlabel("Time ($\\mu$s)", fontsize=20)
        ax.set_ylabel("Drive Strength (MHz)", fontsize=20)
        ax.tick_params(direction='in', length=6, width=2, colors='k', \
                grid_color='r', grid_alpha=0.5, labelsize=14, labelbottom=True, right=True, top=True)
        ax.legend(loc='best', fontsize=12)

    def total_H(self, filename): #adding the pulses to get time dependent Hamiltonian for mesolve
        if self.ROTATING:
            H = [Qobj(self.H_rot())]
            controlHs = self.controlHs()
            fine_pulses = self.return_pulses(filename)
            for index in range(len(fine_pulses)):
                H.append([controlHs[index], fine_pulses[index]]) 
        else: H = 0
        
        return (H)
    
    def run_optimal_control(self, state_transfer=True, 
                            total_time=2000.0, steps=750, max_amp=[2e-4, 2e-4], taylor_terms=None, 
                            convergence={}, reg_coeffs={},
                            plot_only_g=False, states_forbidden_list=[], initial_guess=None, guess_amp = [1e-4, 1e-4],
                            file_name="test", data_path="test"):

        XI = np.kron(self.Q_x, self.I_m)
        YI = np.kron(self.Q_y, self.I_m)
        IX = np.kron(self.I_q, self.M_x)
        IY = np.kron(self.I_q, self.M_y)
        Hops = []
        Hops.extend([XI, YI, IX, IY]) 
        H0 = self.H_rot()
        ops_max_amp = []
        Hnames = []
        qmax_amp = max_amp[0]
        cmax_amp = max_amp[1]
        ops_max_amp.extend([qmax_amp*2*np.pi, qmax_amp*2*np.pi, cmax_amp*2*np.pi, cmax_amp*2*np.pi])
        Hnames.extend(['Qubit_x','Qubit_y','Cavity_x', 'Cavity_y'])
        print([len(Hops), len(ops_max_amp), len(Hnames)])
        U = []
        psi0 = []
        g0 = np.zeros(self.qnum*self.mnum)
        g0[0] = 1.
        e0 = np.zeros(self.qnum*self.mnum)
        e0[self.mnum] = 1.
        psi0.append(g0)
        psi0.append(e0)
        dressed_info = None
        g2 = np.zeros(self.qnum*self.mnum)
        g2[2] = 1.
        g4 = np.zeros(self.qnum*self.mnum)
        g4[4] = 1.
        U.append((g0 + g4)/np.sqrt(2))
        U.append(g2)
        print(U)
        psi0 = np.array(psi0)
        U = np.array(U)
        
        if state_transfer == False:
            U = np.zeros((self.qnum*self.mnum, self.qnum*self.mnum))
            U[1][0] = 1.0
            U[2][1] = 1.0
            U[0][2] = 1.0
            #U[1+self.mnum][self.mnum] = 1
            #U[2+self.mnum][1+self.mnum] = 1
            #U[self.mnum][2+self.mnum] = 1
            for i in range(self.qnum*self.mnum):
                if i not in [0, 1, 2]:
                    U[i][i] = 1
            psi0 = np.array([0, 1, 2])
        #Defining Concerned states (starting states)
        print("starting states:")
        print(psi0)
        print("target states:")
        print(U)

        # #Defining states to include in the drawing of occupation
        if plot_only_g:
            states_draw_list = np.arange(self.mnum)  # want f states
            states_draw_names = []
            for ii in range(self.mnum):
                states_draw_names.append('g_' + str(ii))
        else:
            states_draw_list = np.arange(self.mnum * 2)
            states_draw_names = []
            for ii in range(self.mnum):
                states_draw_names.append('g_' + str(ii))
            for ii in range(self.mnum):
                states_draw_names.append('e_' + str(ii))
            
        states_forbidden_list = states_forbidden_list
        print("states forbidden list: ", states_forbidden_list)
        
        if initial_guess is None and len(guess_amp) > 0: 
            initial_guess = []
            initial_guess.append(np.ones(int(steps))*2*np.pi*(guess_amp[0]))
            initial_guess.append(np.ones(int(steps))*2*np.pi*(guess_amp[0]))
            initial_guess.append(np.ones(int(steps))*2*np.pi*(guess_amp[1]))
            initial_guess.append(np.ones(int(steps))*2*np.pi*(guess_amp[1]))
            initial_guess = np.array(initial_guess)
#         elif initial_guess is not None:
#             t = np.linspace(0, total_time, steps)
#             fine_pulses = []
#             for i in range(self.num_ops):
#                 base_pulse = initial_guess[i][:]  # final control pulse
#                 interpFun = scipy.interpolate.interp1d(self.tlist, base_pulse)
#                 pulse_interp = interpFun(t)
#                 fine_pulses.append(pulse_interp) #nopsxfine_steps matrix containing all the pulses
#             initial_guess = fine_pulses

        #modified Grape to also return the file path
        ss, fp = Grape(H0, Hops, Hnames, U, total_time, steps, psi0, convergence=convergence,
                   draw=[states_draw_list, states_draw_names], state_transfer=state_transfer, use_gpu=False,
                   sparse_H=False, show_plots=True, Taylor_terms=taylor_terms, method='Adam', 
                   initial_guess=initial_guess, maxA=ops_max_amp, reg_coeffs=reg_coeffs, dressed_info=dressed_info, 
                   file_name=file_name, data_path=data_path)
        toterror = ss.rl + ss.l
        return toterror, fp
    
    def plot_optimal_control(self, scales = [4367,4367,81.1684054679128, 81.1684054679128],
                             pad_FFT = 3, filename='', lim_scale=1.0, carrier_freqs=[0,0],
                             fine_steps=None, combine=True, plot_qubit = False, fft_lims=None):
        pulses = self.return_pulses(filename)
        steps = int(self.fine_steps + 1)
        dt = self.dt
        total_time = self.total_time
        base_times = self.base_times
        tlist = self.tlist

        base_pulses = []
        
        f, (ax1, ax2) = plt.subplots(2, 1, figsize=(12,8))

        if plot_qubit: 
            start = 0
        else:
            start = int(2)
        labels = ["Qubit_x", "Qubit_y", "Cavity_x", "Cavity_y"]
        for ii, x in enumerate(pulses[start:]):
            ax1.plot(self.tlist/1e3, (x/2/np.pi)*1e3, label=labels[ii+start]) # plotting the linear frequency in MHz
        ax1.set_xlabel("Time ($\\mu$s)", fontsize=20)
        ax1.set_ylabel("Drive Strength (MHz)", fontsize=20)
        ax1.tick_params(direction='in', length=6, width=2, colors='k', \
                grid_color='r', grid_alpha=0.5, labelsize=14, labelbottom=True, right=True, top=True)
        ax1.legend(loc='best', fontsize=12)
        
        qubit = []
        cavity = []
        qubit.append(pulses[0])
        qubit.append(pulses[1])
        cavity.append(pulses[2])
        cavity.append(pulses[3])

        # plot Fourier transform of control pulses
#         for ii, x in enumerate(pulses[start:]):
            
# #             ax2.plot(1e3*np.fft.fftfreq((2*pad_FFT+1)*steps, d=dt), np.abs(np.fft.fft(np.pad(x,(pad_FFT*steps,pad_FFT*steps),
# #                                                                         'constant'))),'.-', label=labels[ii+start])
#             ax2.plot(1e3*np.fft.fftfreq((2*pad_FFT+1)*steps, d=dt), np.abs(np.fft.fft(np.pad(x,(pad_FFT*steps,pad_FFT*steps),
#                                                                         'constant'))),'.-', label=labels[ii+start])       
        ax2.plot(1e3*np.fft.fftfreq((2*pad_FFT+1)*steps, d=dt), np.abs(np.fft.fft(np.pad(qubit[0]+1j*qubit[1],(pad_FFT*steps, pad_FFT*steps),
                                                                        'constant'))),'.-', label='Qubit')
        ax2.plot(1e3*np.fft.fftfreq((2*pad_FFT+1)*steps, d=dt), np.abs(np.fft.fft(np.pad(cavity[0]+1j*cavity[1],(pad_FFT*steps, pad_FFT*steps),
                                                                        'constant'))),'.-', label='cavity')
        
        if fft_lims:
            ax2.set_xlim(fft_lims[0],fft_lims[1])
        ax2.set_xlabel("Frequency (MHz)", fontsize=20)
        ax2.set_ylabel("FFT", fontsize=20)
        ax2.set_title("Pulse Fourier Transform", fontsize=20)
        ax2.tick_params(direction='in', length=6, width=2, colors='k', \
            grid_color='r', grid_alpha=0.5, labelsize=14, labelbottom=True, right=True, top=True)
        plt.legend(loc='best', fontsize=12)

#         limits = abs(self.hparams["chi"]*2*lim_scale)
        for ii in np.arange(-5, 6, 1):
            ax2.axvline(abs(2*self.hparams['chi'][0])*ii*1e3,color='b',linestyle='dashed')
#         ax2.set_xlim(0,limits)
        plt.tight_layout()        
################################----------------------######################################

    def qutip_mesolve(self, start_state, filename):
        
        psi0 = Qobj(start_state)
        rho0 = psi0*psi0.dag()
        H = self.total_H(filename)

        c_ops = [] #decay operators during the state evolution

        if self.t1params:
            print("Using T1 params")
            kappa_m = 1/self.t1params['T1_m']
            n_thm =  self.t1params['nth_m']
            # lindblad_dissipator in qutip.superoperator; returns the Lindblad master equation with argument 
            c_ops = kappa_m*(1+n_thm)*lindblad_dissipator(self.am) + \
                    kappa_m*(n_thm)*lindblad_dissipator(self.am.dag()) 
            try:
                kappa_q = 1/self.t1params['T1_q']
                n_thq =  self.t1params['nth_q']
                c_ops += kappa_q*(1+ n_thq)*lindblad_dissipator(self.aq) + \
                         kappa_q*(n_thq)*lindblad_dissipator(self.aq.dag())
            except:
                print("No qubit T1 decay params")
        
        if self.t2params:
            print("Using T2 params")
            try:
                gamma_phi = 1/self.t2params['T2_q'] - 1/self.t1params['T1_q']/2.0
                if c_ops == []:
                    c_ops = gamma_phi*lindblad_dissipator(self.sigmaz_q)
                else:
                    c_ops += gamma_phi*lindblad_dissipator(self.sigmaz_q)
            except:
                print("No qubit T2 decay params")
        
        if c_ops == []:
            print("No loss operators")
        
        H0 = Qobj(self.H_rot()) #time independent/ static Hamiltonian
        e_vecs = H0.eigenstates()[1] #[0] returns the eigenvalues
        e_ops = [e_vec*e_vec.dag() for e_vec in e_vecs]
        self.n_cav = np.array([expect(Qobj(np.kron(self.I_q,self.M_z)), e_vec) for e_vec in e_vecs])
        self.n_q = np.array([expect(Qobj(np.kron(self.Q_z,self.I_m)), e_vec) for e_vec in e_vecs])
        
        result_states = mesolve(H, rho0, self.tlist, c_ops=c_ops, e_ops=e_ops)
        return self.tlist, result_states
    
###################################################################################################################
    def plot_output(self, filename, plot_qubit=False, show_e=False, start_states = [0], cmap = 'Reds'):
        
        pulses = self.return_pulses(filename)
        fig, axs = plt.subplots(3, 1, figsize=(14, 12), sharex=True)
        if plot_qubit: 
            start = 0
        else:
            start = int(2)
        labels = ["Qubit_x", "Qubit_y", "Cavity_x", "Cavity_y"]
        for ii, x in enumerate(pulses[start:]):
            axs[0].plot(self.tlist, (x/2/np.pi)*1e3, label=labels[ii+start]) # plotting the linear frequency in MHz
        axs[0].set_ylabel("Drive Strength (MHz)", fontsize=20)
        axs[0].tick_params(direction='in', length=6, width=2, colors='k', \
                grid_color='r', grid_alpha=0.5, labelsize=14, labelbottom=False, right=True, top=True)
        axs[0].legend(loc='best', fontsize=12)
        
        pn_g_final = [] #population at the end of pulse

        """Plotting the evolution of states in the ground state"""
        for state_index in start_states:
            start_state = np.zeros(self.qnum * self.mnum)
            start_state[state_index] = 1.0  # define correct starting state
            print_start_state = start_state
            start_state = Qobj(start_state)
            print("running mesolve for rotating frame")

            tlist_rot, out = self.qutip_mesolve(start_state, filename)
            pops= [out.expect[ii] for ii in np.arange(self.qnum*self.mnum)]
            cutoff = self.qnum*self.mnum
            plot_states = range(cutoff)

            col = cm.tab20c(np.linspace(0, 1, cutoff))
            label = ''
            pn_g = []
            for num in range(cutoff):
                if np.around(self.n_q[num], 1) == 0.0:
                    label = 'g'
                elif np.around(self.n_q[num], 1) == 1.0:
                    label = 'e'
                label = label + ',' + str(int(np.around(self.n_cav[num], 1)))
                if label[:1] == 'g':
                    pn_g.append(pops[num])
                    pn_g_final.append(pops[num][-1])
                if show_e:
                    show_plot = True
                else:
                    if label[:1] == 'g':
                        show_plot = True
                    else:
                        show_plot=False
                if show_plot:
                    axs[1].plot(tlist_rot, pops[num], label=label)
                    axs[1].legend(ncol= 5, prop={'size': 12})                   
#                     print("end", label," pop", pops[num][-1])
                    
                axs[1].set_ylabel(r'$P(n,t|g)$', fontsize=20)
                axs[1].tick_params(direction='in', length=6, width=2, colors='k', \
                grid_color='r', grid_alpha=0.5, labelsize=14, labelbottom=False, right=True, top=True)

            pn_g = np.array(pn_g)
            pn_g_final = np.array(pn_g_final)
            ind = np.argmin(abs(pn_g_final-1.0))
            final_fid = pn_g_final[ind]
            psm = axs[2].pcolormesh(pn_g[::-1],cmap =cmap, vmin = 0, vmax = 1)
            axs[2].set_title("Target state final population: %f"%final_fid)
            axs[2].set_xlabel("Time (ns)", fontsize=20)
            axs[2].set_ylabel("$n$", fontsize=20)
            axs[2].text(50, 15, r'$P(n,t|g)$', fontsize=20)
            axs[2].set_yticks(np.arange(0, self.mnum+1, 5))
            axs[2].tick_params(direction='in', length=6, width=2, colors='k', \
                grid_color='r', grid_alpha=0.5, labelsize=14, labelbottom=True, right=True, top=True)
            plt.show()
            return pn_g_final
print("class definition done")

# System parameters

In [ ]:
#Defining convergence parameters
max_iterations = 700
decay = 350  # max_iterations/2 # amount by which convergence rate is suppressed exponentially in each iteration,
# smaller number is more suppression
# learning rate = (convergence rate) * exp(-(iteration #) / decay)
# found optimal value of about 0.011 from previous LRF run 6-25-19
#chis, kappas in GHZ
# to be safe just have cavity amp < chis
chis = [-0.0011777703723174153, -0.0008619127551763747, -0.0006606630129578532, -0.0005469,
        -0.0004655511487068216, -0.00036840136926547245, -0.0003569580090874056, -0.00031138680426598755,
        -0.00027235167373825496, -0.00027576338150714363]
kappas = [0.0, 9.02e-6, 5.23e-6, 4e-6, 3.24e-6, 1.4e-6, 1.48e-6, 0.72e-6, 0.0, 0.0]
chi_2s = [0, 3.6317466677315835e-6, 1.6492042423932318e-6, 
          1.0194765942550532e-6, 4.142291694098077e-7, 4.4955828071291393e-7, 0, 0, 0, 0]
T1ms = np.array([2.0222, 2.1362, 2.0495, 1.8843, 1.7903, 1.8429, 2.0087, 1.9382,1.9977])*1e3

print("defined")

In [ ]:
def coh_limit(time):
    f = np.exp(-time/t1params["T1_q"])
    return print("Coherence limited fidelity (approx.) = %.4f"%f)

In [ ]:
dwdt_scale = 1.0 * 10 #0.5
dwdt2_scale = 0.1  * 7 #*0.5

# HyperOpt

In [ ]:
#define the system variables
mode = 3
chi_e = chis[mode]
kappa = kappas[mode]
chi_2 = chi_2s[mode]
T1m = T1ms[mode]
hparams = {"chi":[chi_e],"kappa": kappa, "chi_2": [chi_2]}
t1params = {"T1_m": T1m, "nth_m":0.001, "T1_q":79e3, "nth_q":0.015}
t2params = {"T2_q":60e3}
qnum = 2
mnum = 14
# choose a particular mode of the MM cavity 
#mostly use mode 2,3,4
states_forbidden_list = []
qubit_amp = 24  # in MHz
cavity_amp = 29 # in MHz maximum is 1.2 MHz for mode 2 
#pulse_time = 2700
steps = 820
guess_amp = 0.1
states_forbidden_list = [12, 13, 26, 27]
# some of Ankur's pulses labeled with qamp 7.5, camp 0.2, gamp 0.1

target_state = 1

convergence = {'rate': 0.01, 'update_step': 25, 'max_iterations': max_iterations,
               'conv_target': 8e-5, 'learning_rate_decay': decay}
#forbidden_coeff_lists gives the cost function penalty weight for going to forbidden state
reg_coeffs = {'dwdt': dwdt_scale*steps, 'd2wdt2': dwdt2_scale*steps, 
              'forbidden_coeff_list': [10*steps] * len(states_forbidden_list),
              'states_forbidden_list': states_forbidden_list, 'forbid_dressed': False, 'endpoints_zero':10.0}
# expt_name = 'g0_to_g1_SNAP_qamp7.5_camp0.2'
# file_number = 1
# initial_pulse = "S:/_Data/200302 - 3DMM2 cooldown 10 - sideband with LO and mixer/blockade_pulses/" + \
#                 str(file_number).zfill(5) + "_" + expt_name.lower()+".h5"
# initial_pulse = "S:/KevinHe/Optimal Control and Blockade/output_pulses/00006_g0_to_g1_SNAP_qamp0.3_camp0.05_mode0.h5"
# 

In [ ]:
base_guess = 2*np.pi*1e-3 * np.append(np.array([np.ones(steps), np.ones(steps)]) * 0.01, 
                                         np.array([np.ones(steps), np.ones(steps)]) * 0.001).reshape((4, steps))
for j in range(len(base_guess)):
    base_guess[j][0] = 0.0
    base_guess[j][-1] = 0.0

In [ ]:
data_path = 'C:\\Users\\a7b\\Optimal Control\\binom'
file_name = "binomial_SNAP_qamp" + str(round(qubit_amp, 2)) + "_camp" + str(round(cavity_amp, 2)) + "_mode" + str(mode) + "time_" + str(1000)
op = qoc_test_mm(qnum, mnum, hparams, t1params=t1params, t2params=t2params)
initial_guess_rand = 2*np.pi*1e-3 * np.append(np.array([np.random.default_rng().uniform(-qubit_amp, qubit_amp, steps), 
                                                   np.random.default_rng().uniform(-qubit_amp, qubit_amp, steps)]), 
                                                  [np.random.default_rng().uniform(-cavity_amp, cavity_amp, steps), 
                                                   np.random.default_rng().uniform(-cavity_amp, cavity_amp, steps)]).reshape((4, steps))
op.run_optimal_control(state_transfer=True, total_time=2700, 
                       steps=steps, convergence=convergence, max_amp = [qubit_amp*1e-3, cavity_amp*1e-3], 
                       reg_coeffs=reg_coeffs, states_forbidden_list= states_forbidden_list, initial_guess=initial_guess_rand,
                       guess_amp=[], file_name=file_name, data_path=data_path,
                       taylor_terms=[50, 10])

In [ ]:
data_path = 'C:\\Users\\a7b\\Optimal Control\\binom'
file_name = "binomial_SNAP_qamp" + str(round(qubit_amp, 2)) + "_camp" + str(round(cavity_amp, 2)) + "_mode" + str(mode) + "time_" + str(1000)
op = qoc_test_mm(qnum, mnum, hparams, t1params=t1params, t2params=t2params)
initial_guess_rand = 2*np.pi*1e-3 * np.append(np.array([np.random.default_rng().uniform(-qubit_amp, qubit_amp, steps), 
                                                   np.random.default_rng().uniform(-qubit_amp, qubit_amp, steps)]), 
                                                  [np.random.default_rng().uniform(-cavity_amp, cavity_amp, steps), 
                                               np.random.default_rng().uniform(-cavity_amp, cavity_amp, steps)]).reshape((4, steps))
op.run_optimal_control(state_transfer=True, total_time=2700, 
                       steps=steps, convergence=convergence, max_amp = [qubit_amp*1e-3, cavity_amp*1e-3], 
                       reg_coeffs=reg_coeffs, states_forbidden_list= states_forbidden_list, initial_guess=initial_guess_rand,
                       guess_amp=[], file_name=file_name, data_path=data_path,
                       taylor_terms=[50, 10])

In [ ]:
#define the system variables
mode = 3
chi_e = chis[mode]
kappa = kappas[mode]
chi_2 = chi_2s[mode]
T1m = T1ms[mode]
hparams = {"chi":[chi_e],"kappa": kappa, "chi_2": [chi_2]}
t1params = {"T1_m": T1m, "nth_m":0.001, "T1_q":79e3, "nth_q":0.015}
t2params = {"T2_q":60e3}
qnum = 2
mnum = 12
# choose a particular mode of the MM cavity 
#mostly use mode 2,3,4
states_forbidden_list = []
qubit_amp = 24  # in MHz
cavity_amp = 29 # in MHz maximum is 1.2 MHz for mode 2 
#pulse_time = 2700
steps = 100
guess_amp = 0.1
states_forbidden_list = []
# some of Ankur's pulses labeled with qamp 7.5, camp 0.2, gamp 0.1

target_state = 1

convergence = {'rate': 0.01, 'update_step': 25, 'max_iterations': max_iterations,
               'conv_target': 8e-5, 'learning_rate_decay': decay}
#forbidden_coeff_lists gives the cost function penalty weight for going to forbidden state
reg_coeffs = {'dwdt': dwdt_scale*steps, 'd2wdt2': dwdt2_scale*steps, 
              'forbidden_coeff_list': [10*steps] * len(states_forbidden_list),
              'states_forbidden_list': states_forbidden_list, 'forbid_dressed': False, 'endpoints_zero':10.0}
# expt_name = 'g0_to_g1_SNAP_qamp7.5_camp0.2'
# file_number = 1
# initial_pulse = "S:/_Data/200302 - 3DMM2 cooldown 10 - sideband with LO and mixer/blockade_pulses/" + \
#                 str(file_number).zfill(5) + "_" + expt_name.lower()+".h5"
# initial_pulse = "S:/KevinHe/Optimal Control and Blockade/output_pulses/00006_g0_to_g1_SNAP_qamp0.3_camp0.05_mode0.h5"
# 
data_path = 'C:\\Users\\a7b\\Optimal Control\\binom'

file_name = "binomial_SNAP_qamp" + str(round(qubit_amp, 2)) + "_camp" + str(round(cavity_amp, 2)) + "_mode" + str(mode) + "time_" + str(1000)
op = qoc_test_mm(qnum, mnum, hparams, t1params=t1params, t2params=t2params)
initial_guess_rand = 2*np.pi*1e-3 * np.append(np.array([np.random.default_rng().uniform(-qubit_amp, qubit_amp, steps), 
                                                   np.random.default_rng().uniform(-qubit_amp, qubit_amp, steps)]), 
                                                  [np.random.default_rng().uniform(-cavity_amp, cavity_amp, steps), 
                                               np.random.default_rng().uniform(-cavity_amp, cavity_amp, steps)]).reshape((4, steps))
op.run_optimal_control(state_transfer=True, total_time=2700, 
                       steps=steps, convergence=convergence, max_amp = [qubit_amp*1e-3, cavity_amp*1e-3], 
                       reg_coeffs=reg_coeffs, states_forbidden_list= states_forbidden_list, initial_guess=initial_guess_rand,
                       guess_amp=[], file_name=file_name, data_path=data_path,
                       taylor_terms=[10, 30])